In [ ]:
import urllib.request as req
import os.path, random
import json
import requests

key = '9bd94b7e582fdba45a601ed68f788300'
url = 'http://kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json?key=9bd94b7e582fdba45a601ed68f788300&movieNm=쥬라기'

response = requests.get(url)

# 상태코드 200이면 정상 응답
if response.status_code == 200:
    data = response.json()  # JSON을 Python dict로 변환
    print(data)  # 전체 데이터 출력

    # 예: 영화 리스트 출력
    for movie in data["movieListResult"]["movieList"]:
        print(f"제목: {movie['movieNm']}, 장르: {movie['genreAlt']}")
else:
    print("요청 실패:", response.status_code)

# savename = "movieDB.json"

# # exists를 이용하여 경로에 해당하는 파일이 없으면
# # if문을 실행하여 url의 내용으로 파일 생성
# if not os.path.exists(savename):
#     req.urlretrieve(url, savename)

# # json 파일 분석
# items = json.load(open(savename, 'r', encoding='utf-8'))

# # s = oprn(savename, 'r', encoding*('utf-2).read()

{'movieListResult': {'totCnt': 39, 'source': '영화진흥위원회', 'movieList': [{'movieCd': '20252527', 'movieNm': '쥬라기 월드: 새로운 시작', 'movieNmEn': 'Jurassic World: Rebirth', 'prdtYear': '2025', 'openDt': '20250702', 'typeNm': '장편', 'prdtStatNm': '개봉', 'nationAlt': '미국', 'genreAlt': '어드벤처,액션,스릴러,공포(호러)', 'repNationNm': '미국', 'repGenreNm': '어드벤처', 'directors': [{'peopleNm': '가렛 에드워즈'}], 'companys': []}, {'movieCd': '2025A109', 'movieNm': '쥬라기 아일랜드', 'movieNmEn': 'Jurassic Park', 'prdtYear': '2024', 'openDt': '', 'typeNm': '장편', 'prdtStatNm': '기타', 'nationAlt': '중국', 'genreAlt': '액션', 'repNationNm': '중국', 'repGenreNm': '액션', 'directors': [], 'companys': []}, {'movieCd': '20242021', 'movieNm': '쥬라기캅스 극장판: 전설의 고대생물을 찾아라', 'movieNmEn': '', 'prdtYear': '2024', 'openDt': '20240815', 'typeNm': '장편', 'prdtStatNm': '개봉', 'nationAlt': '한국', 'genreAlt': '애니메이션', 'repNationNm': '한국', 'repGenreNm': '애니메이션', 'directors': [{'peopleNm': '이재민'}], 'companys': [{'companyCd': '20229242', 'companyNm': '(주)스튜디오버튼 '}]}, 

In [ ]:
# 추천 코드

import requests  # API 요청을 보내기 위한 라이브러리

# 영화진흥위원회 API Key와 기본 URL
API_KEY = "9bd94b7e582fdba45a601ed68f788300"
BASE_URL = "http://kobis.or.kr/kobisopenapi/webservice/rest/movie"

# 🔍 [1단계] 영화 제목으로 영화 검색
def search_movie_by_name(movie_name):
    url = f"{BASE_URL}/searchMovieList.json"  # 영화 검색용 API 엔드포인트
    params = {
        "key": API_KEY,
        "movieNm": movie_name  # 검색할 영화 제목
    }

    # API 요청 보내기
    response = requests.get(url, params=params)
    
    # 응답 JSON 데이터를 딕셔너리 형태로 변환
    data = response.json()
    
    # 검색된 영화 리스트 가져오기
    movie_list = data.get("movieListResult", {}).get("movieList", [])
    
    return movie_list

# 🎯 [2단계] 장르 기반으로 비슷한 영화 추천
def get_similar_movies_by_genre(genre_keyword, exclude_movie_cd=None, count=5):
    url = f"{BASE_URL}/searchMovieList.json"  # 동일한 API 사용
    params = {
        "key": API_KEY,
        "itemPerPage": 100,          # 최대 100개 영화 검색
        "openStartDt": "20000101"    # 2000년 이후 개봉작만 대상으로 필터링
    }

    # API 요청 보내기
    response = requests.get(url, params=params)
    
    # 응답을 JSON으로 파싱
    data = response.json()
    
    # 전체 영화 리스트
    all_movies = data.get("movieListResult", {}).get("movieList", [])

    # 추천 영화 저장 리스트
    similar_movies = []

    # 전체 영화 중에서 장르가 비슷한 영화 찾기
    for movie in all_movies:
        genres = movie.get("genreAlt", "")  # 해당 영화의 장르 정보 가져오기

        # 입력한 장르 키워드가 포함되어 있고, 자기 자신은 제외할 경우
        if genre_keyword in genres:
            if exclude_movie_cd and movie["movieCd"] == exclude_movie_cd:
                continue  # 동일한 영화 코드면 스킵

            # 제목과 장르를 튜플로 저장
            similar_movies.append((movie["movieNm"], genres))

        # 추천 개수만큼 모이면 종료
        if len(similar_movies) >= count:
            break

    return similar_movies

# 🧪 [실행 부분] 사용자 입력 받고 추천 시작
# ---------------------------------------------
search_title = input("검색할 영화 제목을 입력하세요: ")

# [1] 입력한 제목으로 영화 검색
movies_found = search_movie_by_name(search_title)

# 검색 결과가 없을 경우 처리
if not movies_found:
    print("해당 영화가 검색되지 않았습니다.")
else:
    # 첫 번째 검색 결과를 기준으로 추천 시작
    main_movie = movies_found[0]  # 여러 개일 경우 첫 번째만 사용
    movie_cd = main_movie["movieCd"]  # 영화 고유 ID
    movie_title = main_movie["movieNm"]  # 영화 제목
    genre = main_movie.get("genreAlt", "알 수 없음")  # 장르 정보

    print(f"\n[검색 결과] '{movie_title}' (장르: {genre})")

    # [2] 장르 기반 추천
    if genre != "알 수 없음":
        # 장르가 여러 개일 경우 첫 번째 장르만 사용해서 추천
        genre_keyword = genre.split(",")[0]
        
        # 장르 키워드를 기준으로 비슷한 영화 5개 추천 받기
        recommendations = get_similar_movies_by_genre(
            genre_keyword,
            exclude_movie_cd=movie_cd
        )

        print(f"\n[추천 영화] '{movie_title}'와(과) 비슷한 장르의 영화:")
        for i, (rec_title, rec_genre) in enumerate(recommendations, 1):
            print(f"{i}. {rec_title} (장르: {rec_genre})")
    else:
        print("해당 영화의 장르 정보를 찾을 수 없습니다.")

In [ ]:
# 최근 1000편 영화 json형태로 저장

import requests
import json
from time import sleep

API_KEY = "9bd94b7e582fdba45a601ed68f788300"
BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"

def fetch_movies(api_key, max_count=1000):
    movies = []
    per_page = 100
    total_pages = max_count // per_page

    for page in range(1, total_pages + 1):
        params = {
            "key": api_key,
            "curPage": str(page),
            "itemPerPage": str(per_page),
            "openStartDt": "2015",  # 최근 10년간 개봉작으로 설정
            "openEndDt": "2025"
        }

        response = requests.get(BASE_URL, params=params)
        data = response.json()

        movie_list = data.get("movieListResult", {}).get("movieList", [])
        if not movie_list:
            print(f"{page} 페이지에 영화가 없습니다. 수집 종료.")
            break

        for m in movie_list:
            title = m.get("movieNm", "")
            open_date = m.get("openDt", "")
            # 감독명은 directors 배열 안에 peopleNm 필드 여러개일 수 있음
            directors = ", ".join([d.get("peopleNm", "") for d in m.get("directors", [])]) if m.get("directors") else "정보없음"
            genre = m.get("genreAlt", "")

            movies.append({
                "title": title,
                "open_date": open_date,
                "directors": directors,
                "genre": genre
            })

            if len(movies) >= max_count:
                print(f"{max_count}편의 영화 데이터를 수집했습니다.")
                return movies

        print(f"{page} 페이지 완료. 누적 {len(movies)}편.")

        # API 부하 줄이기 위해 잠시 쉬기
        sleep(0.5)

    return movies

if __name__ == "__main__":
    movies_data = fetch_movies(API_KEY, max_count=1000)
    with open("movies_DB.json", "w", encoding="utf-8") as f:
        json.dump(movies_data, f, ensure_ascii=False, indent=2)
    print(f"총 {len(movies_data)}편의 영화 정보를 movies_DB.json에 저장했습니다.")

1 페이지 완료. 누적 100편.
2 페이지 완료. 누적 200편.
3 페이지 완료. 누적 300편.
4 페이지 완료. 누적 400편.
5 페이지 완료. 누적 500편.
6 페이지 완료. 누적 600편.
7 페이지 완료. 누적 700편.
8 페이지 완료. 누적 800편.
9 페이지 완료. 누적 900편.
1000편의 영화 데이터를 수집했습니다.
총 1000편의 영화 정보를 movies_DB.json에 저장했습니다.


In [4]:
import requests
import json
from time import sleep
from datetime import datetime

API_KEY = "9bd94b7e582fdba45a601ed68f788300"
BASE_URL = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieList.json"

def get_total_count(api_key):
    params = {
        "key": api_key,
        "curPage": "1",
        "itemPerPage": "1",
        "openStartDt": "1990",
        "openEndDt": str(datetime.now().year)
    }
    response = requests.get(BASE_URL, params=params)
    data = response.json()
    total_count = data.get("movieListResult", {}).get("totCnt", 0)
    print(f"1990년 이후 총 영화 수: {total_count}편")
    return total_count

def fetch_movies_1990s_onward(api_key):
    total_count = get_total_count(api_key)
    per_page = 100
    total_pages = (total_count // per_page) + (1 if total_count % per_page else 0)
    print(f"총 {total_pages} 페이지를 조회합니다.")

    all_movies = []

    for page in range(1, total_pages + 1):
        params = {
            "key": api_key,
            "curPage": str(page),
            "itemPerPage": str(per_page),
            "openStartDt": "1990",
            "openEndDt": str(datetime.now().year)
        }
        response = requests.get(BASE_URL, params=params)
        data = response.json()

        movie_list = data.get("movieListResult", {}).get("movieList", [])
        if not movie_list:
            print(f"{page} 페이지에 영화 데이터가 없습니다. 중단합니다.")
            break

        for m in movie_list:
            movie_code = m.get("movieCd", "")
            title_ko = m.get("movieNm", "")
            title_en = m.get("movieNmEn", "")
            open_date = m.get("openDt", "")
            directors = ", ".join([d.get("peopleNm", "") for d in m.get("directors", [])]) if m.get("directors") else "정보없음"
            genre = m.get("genreAlt", "")

            all_movies.append({
                "movie_code": movie_code,
                "title_ko": title_ko,
                "title_en": title_en,
                "open_date": open_date,
                "directors": directors,
                "genre": genre
            })

        print(f"{page} 페이지 완료. 누적 {len(all_movies)}편")
        sleep(0.3)

    return all_movies

if __name__ == "__main__":
    movies = fetch_movies_1990s_onward(API_KEY)
    with open("movies_1990_onward.json", "w", encoding="utf-8") as f:
        json.dump(movies, f, ensure_ascii=False, indent=2)
    print(f"총 {len(movies)}편의 영화 정보를 movies_1990_onward.json에 저장했습니다.")

1990년 이후 총 영화 수: 28876편
총 289 페이지를 조회합니다.
1 페이지 완료. 누적 100편
2 페이지 완료. 누적 200편
3 페이지 완료. 누적 300편
4 페이지 완료. 누적 400편
5 페이지 완료. 누적 500편
6 페이지 완료. 누적 600편
7 페이지 완료. 누적 700편
8 페이지 완료. 누적 800편
9 페이지 완료. 누적 900편
10 페이지 완료. 누적 1000편
11 페이지 완료. 누적 1100편
12 페이지 완료. 누적 1200편
13 페이지 완료. 누적 1300편
14 페이지 완료. 누적 1400편
15 페이지 완료. 누적 1500편
16 페이지 완료. 누적 1600편
17 페이지 완료. 누적 1700편
18 페이지 완료. 누적 1800편
19 페이지 완료. 누적 1900편
20 페이지 완료. 누적 2000편
21 페이지 완료. 누적 2100편
22 페이지 완료. 누적 2200편
23 페이지 완료. 누적 2300편
24 페이지 완료. 누적 2400편
25 페이지 완료. 누적 2500편
26 페이지 완료. 누적 2600편
27 페이지 완료. 누적 2700편
28 페이지 완료. 누적 2800편
29 페이지 완료. 누적 2900편
30 페이지 완료. 누적 3000편
31 페이지 완료. 누적 3100편
32 페이지 완료. 누적 3200편
33 페이지 완료. 누적 3300편
34 페이지 완료. 누적 3400편
35 페이지 완료. 누적 3500편
36 페이지 완료. 누적 3600편
37 페이지 완료. 누적 3700편
38 페이지 완료. 누적 3800편
39 페이지 완료. 누적 3900편
40 페이지 완료. 누적 4000편
41 페이지 완료. 누적 4100편
42 페이지 완료. 누적 4200편
43 페이지 완료. 누적 4300편
44 페이지 완료. 누적 4400편
45 페이지 완료. 누적 4500편
46 페이지 완료. 누적 4600편
47 페이지 완료. 누적 4700편
48 페이지 완료. 누적 4800편
49 페이지 완료. 누적 49

In [1]:
import json
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D

# 1. 데이터 로드
with open("movies_1990_onward.json", encoding="utf-8") as f:
    movies = json.load(f)

# 2. 텍스트 특징: 감독명 + 장르
texts = [f"{m['directors']} {m['genre']}" for m in movies]

# 3. Tokenizer로 정수 시퀀스로 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len, padding='post')

# 4. Keras 모델 정의 (학습 없이 벡터 추출만)
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 50

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    GlobalAveragePooling1D()
])

# 5. 임베딩 벡터 추출
embeddings = model.predict(X, verbose=0)

# 6. 저장: movies + tokenizer + embeddings + max_len
with open("movie_recommendation.pkl", "wb") as f:
    pickle.dump({
        "movies": movies,
        "tokenizer": tokenizer,
        "embeddings": embeddings,
        "max_len": max_len
    }, f)

print("저장 완료: movie_recommend.pkl")

C:\Users\user\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


저장 완료: movie_recommend.pkl


In [7]:
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.preprocessing.sequence import pad_sequences

def recommend_movie(title_query, top_n=5):
    # 저장된 데이터 로드
    with open("movie_recommendation.pkl", "rb") as f:
        data = pickle.load(f)

    movies = data["movies"]
    tokenizer = data["tokenizer"]
    embeddings = data["embeddings"]
    max_len = data["max_len"]

    # 입력 제목과 일치하는 영화 찾기
    idx = next((i for i, m in enumerate(movies)
                if title_query.lower() in m["title_ko"].lower()
                or title_query.lower() in m["title_en"].lower()), None)

    if idx is None:
        print(f"❌ '{title_query}' 제목의 영화를 찾을 수 없습니다.")
        return

    # 유사도 계산
    target_vector = embeddings[idx].reshape(1, -1)
    sim_scores = cosine_similarity(target_vector, embeddings)[0]

    # 가장 유사한 top_n개 추천 (자기 자신 제외)
    similar_indices = sim_scores.argsort()[::-1][1:top_n + 1]

    print(f"\n🎬 '{movies[idx]['title_ko']}'와 비슷한 영화 추천:")
    for i in similar_indices:
        print(f"- {movies[i]['title_ko']} ({movies[i]['title_en']}) | 장르: {movies[i]['genre']} | 감독 : {movies[i]['directors']} | 개봉일: {movies[i]['open_date']}")

# 예시 실행
recommend_movie("어벤져스: 엔드게임", top_n=5)


🎬 '어벤져스: 엔드게임'와 비슷한 영화 추천:
- 캡틴 아메리카: 시빌 워 (Captain America: Civil War) | 장르: 액션,SF,스릴러 | 감독 : 안소니 루소, 조 루소 | 개봉일: 20160427
- 캡틴 아메리카: 윈터 솔져 (Captain America: The Winter Soldier) | 장르: 액션,어드벤처,SF | 감독 : 조 루소, 안소니 루소 | 개봉일: 20140326
- 그레이 맨 (The Gray Man) | 장르: 액션,스릴러 | 감독 : 안소니 루소, 조 루소 | 개봉일: 20220713
- 웰컴 투 콜린우드 (Welcome To Collinwood) | 장르: 코미디,범죄 | 감독 : 안소니 루소, 조 루소 | 개봉일: 20030117
- 어벤져스: 인피니티 워 (Avengers: Infinity War) | 장르: 액션,어드벤처,판타지,SF | 감독 : 안소니 루소, 조 루소 | 개봉일: 20180425


In [11]:
import requests
import json
from time import sleep
from datetime import datetime

TMDB_API_KEY = '5349acaed8d7c3af060172341881019e' # TMDbd의 API 키
TMDB_BASE = "https://api.themoviedb.org/3" # TMDb API의 기본 url
MAX_MOVIES = 10000 # 최근 기준으로 영화 10000편을 추출하기 위한 값
MOVIES_PER_PAGE = 20  # TMDb API 기본 페이지당 20개의 영화를 보여줌
MAX_PAGES = MAX_MOVIES // MOVIES_PER_PAGE  # 최대 페이지 수 (500) == (20 * 500 = 10000)

all_movies = [] # json의 저장할 리스트
today_str = datetime.now().strftime("%Y-%m-%d") # 현재(07_15)기준 검색을 위한 날짜
# 감독(디렉터)는 다른 페이지에 존재하므로 다른 url을 이용
def get_director(movie_id):
    url = f"{TMDB_BASE}/movie/{movie_id}/credits"
    params = {"api_key": TMDB_API_KEY}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return ""
    data = res.json()
    directors = [c["name"] for c in data.get("crew", []) if c["job"] == "Director"]
    return directors[0] if directors else ""

for page in range(1, MAX_PAGES + 1):
    params = {
        "api_key": TMDB_API_KEY,
        "language": "ko",
        "sort_by": "primary_release_date.desc",  # 최신순 내림차순
        "include_adult": False,  # 성인물 제외
        "page": page,
        "primary_release_date.lte": today_str  # 오늘 이전 개봉작만
    }

    response = requests.get(f"{TMDB_BASE}/discover/movie", params=params)
    if response.status_code != 200:
        print(f"페이지 {page} 요청 실패: {response.status_code}")
        break

    data = response.json()
    movies = data.get("results", [])
    if not movies:
        print(f"페이지 {page}에 데이터 없음, 중단")
        break

    for m in movies:
        movie_id = m.get("id")
        if not movie_id:
            continue

        director = get_director(movie_id)

        # 임시로 genre_ids 저장
        genre_ids = m.get("genre_ids", [])

        # 아래에서 genre_id 리스트를 genre_name 리스트로 변환
        movie_info = {
            "movie_id": movie_id,
            "title_ko": m.get("title", ""),
            "title_en": m.get("original_title", ""),
            "overview": m.get("overview", ""),
            "genres": genre_ids,  # 여기서는 id 리스트 그대로 저장
            "director": director,
            "vote_average": m.get("vote_average", 0),
            "release_date": m.get("release_date", ""),
            "poster_path": m.get("poster_path", "")
        }
        all_movies.append(movie_info)

    print(f"페이지 {page} 완료, 누적 영화 수: {len(all_movies)}편")

    if len(all_movies) >= MAX_MOVIES:
        print("최대 영화 수 도달, 중단")
        break

    sleep(0.3)  # API 제한 고려

# 장르 ID → 이름 매핑
def get_genre_mapping():
    url = f"{TMDB_BASE}/genre/movie/list"
    params = {"api_key": TMDB_API_KEY, "language": "ko"}
    res = requests.get(url, params=params)
    if res.status_code != 200:
        return {}
    data = res.json()
    return {g['id']: g['name'] for g in data.get('genres', [])} # 장르를 id로 나타내서 그에 해당하는 장르값을 구해서 딕셔너리로 반환

genre_map = get_genre_mapping()
for m in all_movies:
    genre_ids = m.get("genres", [])
    m["genres"] = [genre_map.get(gid, "") for gid in genre_ids]

with open("movies_DB.json", "w", encoding="utf-8") as f:
    json.dump(all_movies, f, ensure_ascii=False, indent=2)

print(f"✅ 저장 완료: 총 {len(all_movies)}편 → movies_DB.json")

페이지 1 완료, 누적 영화 수: 20편
페이지 2 완료, 누적 영화 수: 40편
페이지 3 완료, 누적 영화 수: 60편
페이지 4 완료, 누적 영화 수: 80편
페이지 5 완료, 누적 영화 수: 100편
페이지 6 완료, 누적 영화 수: 120편
페이지 7 완료, 누적 영화 수: 140편
페이지 8 완료, 누적 영화 수: 160편
페이지 9 완료, 누적 영화 수: 180편
페이지 10 완료, 누적 영화 수: 200편
페이지 11 완료, 누적 영화 수: 220편
페이지 12 완료, 누적 영화 수: 240편
페이지 13 완료, 누적 영화 수: 260편
페이지 14 완료, 누적 영화 수: 280편
페이지 15 완료, 누적 영화 수: 300편
페이지 16 완료, 누적 영화 수: 320편
페이지 17 완료, 누적 영화 수: 340편
페이지 18 완료, 누적 영화 수: 360편
페이지 19 완료, 누적 영화 수: 380편
페이지 20 완료, 누적 영화 수: 400편
페이지 21 완료, 누적 영화 수: 420편
페이지 22 완료, 누적 영화 수: 440편
페이지 23 완료, 누적 영화 수: 460편
페이지 24 완료, 누적 영화 수: 480편
페이지 25 완료, 누적 영화 수: 500편
페이지 26 완료, 누적 영화 수: 520편
페이지 27 완료, 누적 영화 수: 540편
페이지 28 완료, 누적 영화 수: 560편
페이지 29 완료, 누적 영화 수: 580편
페이지 30 완료, 누적 영화 수: 600편
페이지 31 완료, 누적 영화 수: 620편
페이지 32 완료, 누적 영화 수: 640편
페이지 33 완료, 누적 영화 수: 660편
페이지 34 완료, 누적 영화 수: 680편
페이지 35 완료, 누적 영화 수: 700편
페이지 36 완료, 누적 영화 수: 720편
페이지 37 완료, 누적 영화 수: 740편
페이지 38 완료, 누적 영화 수: 760편
페이지 39 완료, 누적 영화 수: 780편
페이지 40 완료, 누적 영화 수: 800편
페이지 41 완료, 누적

In [14]:
import json
import re

# 한글 포함 여부 체크 함수
def contains_korean(text):
    if text is None:
        return False
    return bool(re.search('[가-힣]', text))

# 기존 JSON 파일 불러오기
with open("movies_DB.json", encoding="utf-8") as f:
    movies = json.load(f)

# 한글이 포함된 title_ko만 필터링
movies_with_korean_title = [movie for movie in movies if contains_korean(movie.get("title_ko", ""))]

print(f"한글이 포함된 제목의 영화 수: {len(movies_with_korean_title)}")

# 새 JSON 파일로 저장
with open("movies_with_korean_title.json", "w", encoding="utf-8") as f:
    json.dump(movies_with_korean_title, f, ensure_ascii=False, indent=2)

print("한글 제목이 포함된 영화 데이터만 저장 완료.")

한글이 포함된 제목의 영화 수: 540
한글 제목이 포함된 영화 데이터만 저장 완료.
